# Direct Radio Modem Test

In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
import struct as pystruct
from binascii import hexlify
import sys
import os.path
import datetime
!umask 2

In [ ]:
sys.path.insert(0,"../tagnet")
from tagnet.tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnet.tagnames import TagName
from tagnet.tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
sys.path.insert(0,"../tagfuse")
sys.path.insert(0,"../si446x/")
from tagfuse.radioutils import payload2values, radio_receive_msg, radio_send_msg
from tagfuse.radioutils import radio_start, radio_show_config, radio_poll
from tagfuse.radioutils import radio_get_rtctime, radio_set_rtctime

In [ ]:
from tagfuse.tagfuseargs import process_cmd_args, get_cmd_args
import tagfuse.tagfuseargs
tagfuse.tagfuseargs.global_args
tagfuse.tagfuseargs.global_args=None
#sys.argv = ['tagfuse.py', '~/tags', '--disable_sparse']
sys.argv = ['tagfuse.py', '/home/pi/tag_store']
process_cmd_args()
get_cmd_args()
from tagfuse.TagFuseTree import TagFuseTagTree

In [ ]:
sys.path

In [ ]:
from tagfuse.myversion import __version__
'tagfuse', __version__

In [ ]:
print('Test Start Time: {}'.format(datetime.datetime.now()))
#print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
#radio = radio_start()
from si446x import Si446xRadio
from si446x import get_config_wds, get_name_wds, wds_default_config
radio=Si446xRadio(0)
if (radio == None):
    raise RuntimeError('radio_start: could not instantiate radio')
radio.unshutdown()
wds_default_config(0) # set index to force alternate default config
radio.write_config()
radio.config_frr()
radio.set_property('PKT', 0x0b, '\x28\x28') # tx/rx threshold
radio.set_property('PREAMBLE', 0, '\x40')   # long preamble

In [ ]:
radio_show_config(radio.dump_radio())

In [ ]:
get_name_wds()

In [ ]:
import RPi.GPIO as GPIO

In [ ]:
GPIO.input(radio.gpio_cts)

In [ ]:
#radio.spi.hard_cts

## Set Defaults

In [ ]:
# default paramters
MAX_WAIT            = 1000
MAX_RECV            = 2
MAX_PAYLOAD         = 254
MAX_RETRIES         = 3
RADIO_POWER         = 20
SHORT_DELAY         = 1000

In [ ]:
ITERATIONS = 200000
SEND_DELAY=.04
pwr = 20
msg = bytearray('0000' + ('0123456789' * 25 + '11'))
msg[0] = len(msg) - 1

In [ ]:
radio.set_power(RADIO_POWER)

In [ ]:
STOP

## Send Packets

In [ ]:
print('start', datetime.datetime.now())
for i in range(ITERATIONS):
    sz = i % MAX_PAYLOAD
    if sz < 4: sz = 4
    msg[1] = i % 256
#    msg[0] = sz
    msg[0] = 246
    radio_send_msg(radio, msg[:sz+1], tag_pwr)
    sleep(SEND_DELAY)
print('end', datetime.datetime.now(), i)

In [ ]:
radio.trace.display(radio.trace.filter(count=-36))

In [ ]:
import monotonic
monotonic.micros()

## Listen for packets

In [ ]:
STOP

In [ ]:
fails = 0
last = 0
total_bytes = 0
show_status = 10
last_fail = False
start = True
rpsmsg = bytearray()
print('start', datetime.datetime.now())
for i in range(ITERATIONS):
    rspmsg, rssi, status = radio_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if rspmsg:
        total_bytes += len(rspmsg)
        reqset = set(msg[4:])
        rspset = set(rspmsg[4:])
        # zzz print(hexlify(rspmsg), reqset, rspset)
        if rspset > reqset:
            print('data error', reqset, rspset, ' ' * 10)
            break
        for x in range(4, len(rspmsg[4:])-4):
            if msg[x] != rspmsg[x]:
                print('data error', x, msg[x], rspmsg[x], ' ' * 6)
        if (((rspmsg[0] + 1) != len(rspmsg)) or (rspmsg[1] != last % 256)):
            last = rspmsg[1] + 1
            if start:
                start = False
            else:
                fails += 1
                print('{}, error, i:{}, fails:{}, rssi:{}, len:{}/{}, seq:{}, last:{}{}'.format(datetime.datetime.now(),
                                                                                                i, fails, rssi,
                                                                                                rspmsg[0]+1,
                                                                                                len(rspmsg),
                                                                                                rspmsg[1],
                                                                                                last,
                                                                                                ' '*10), end='\r')
        else:
            if (i % show_status) == 0:
                print('{}, status, i:{}, fails:{}, rssi:{}, total:{}, len:{}/{}, seq:{}, last:{}{}'.format(datetime.datetime.now(),
                                                                                                i, fails, rssi,
                                                                                                total_bytes,
                                                                                                rspmsg[0]+1,
                                                                                                len(rspmsg),
                                                                                                rspmsg[1],
                                                                                                last, ' '*10), end='\r')
            last = (last + 1) % 256
        last_fail = False
    else:
        fails += 1
        if not last_fail:
            print('{}, tmout, i:{}, fails:{}{}'.format(datetime.datetime.now(), i, fails, ' '*10), end='\r')
            last_fail = True
        continue
print('{}, error, i:{}, fails:{}, rssi:{}, total:{}, len:{}/{}, seq:{}, last:{}    '.format(datetime.datetime.now(),
                                                                                        i, fails, rssi,
                                                                                        total_bytes,
                                                                                        rspmsg[0]+1,
                                                                                        len(rspmsg),
                                                                                        rspmsg[1],
                                                                                        last,
                                                                                        ' '*10),end='\r')

In [ ]:
print(len(rspmsg), hexlify(rspmsg))
print('---',x)
print(len(msg), hexlify(msg))

In [ ]:
help(rspset)

In [ ]:
print(msg[15], rspmsg[15], msg[15] != rspmsg[15])

In [ ]:
radio_receive_msg(radio, MAX_RECV, 10)

In [ ]:
hexlify(msg)

In [ ]:
status

In [ ]:
i, fails, len(msg), msg[0], last, msg[1], msg

## Functions for Accessing Internal Radio State

In [ ]:
STOP

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from si446xdef import *


def si446x_device_group_fetch_and_decode(group):
    gname = radio_config_group_ids.decoding[group]
    g_s = radio_config_groups[radio_config_group_ids.build(gname)]
    gid = radio_config_group_ids.build(gname)
    p = si446x_device_get_property(radio, gname, 0, g_s.sizeof())
    print(g_s, insert_space(p))
    #print(gname, len(p), hexlify(p))
    print(radio_display_structs[g_s](g_s, p))
    return None


def si446x_device_command_fetch_and_decode(cmd):
    cname = radio_status_cmd_ids.decoding[cmd]
    cfunc, cstr = radio_status_commands[radio_config_cmd_ids.build(cname)]
#    print(cname, cmd)
    if (cfunc):
        cmd = cfunc(cname)
        if (cmd):
#            print(cfunc, hexlify(cmd), cstr)
            radio.spi.command(cmd, cstr)
            rsp = radio.spi.response(cstr.sizeof(), cstr.name)
            if (rsp):
#                print(cstr, radio_display_structs[cstr])
                print(cstr, insert_space(rsp))
                print(radio_display_structs[cstr](cstr, rsp))
            else:
                print('no response')
        else:
            print('no command')
    else:
        print('no function')
    return None

def si446x_device_get_property(radio, g_n, start, limit):
    prop_x = 0
    prop_b = bytearray()
    while (prop_x < limit):
        chunk_size = limit - prop_x
        x = MAX_RADIO_RSP if (chunk_size >= MAX_RADIO_RSP) else chunk_size
        rsp = radio.get_property(g_n, prop_x, x)
        if (rsp):
            prop_b += bytearray(rsp[:x])
            prop_x += x
        else:
            return None
    return prop_b

## Interactive Group Properties

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
STOP

In [ ]:
import time
ix = 100
tt = time.time()
for i in range(ix): x = 0
d_tt = time.time() - tt
ct = time.clock()
for i in range(ix): y = 0
d_ct = time.clock() - ct
d_ct/num_iters, d_tt/num_iters

In [ ]:
import si446xcfg as cfg

In [ ]:
help(cfg)

In [ ]:
cfg.set_real_time()

In [ ]:
p=radio.get_gpio()
p

In [ ]:
for k,v in p.iteritems():
    try:
        print(v.mode)
    except:
        continue

In [ ]:
radio.config_gpio(gpio2='RX_STATE', gpio3='TX_STATE')

In [ ]:
for i in range(200000):
    msg, rssi, status = radio_receive_msg(radio, MAX_RECV, 10000)
    if msg:
        print('{} {}/{} {}'.format(datetime.datetime.now(), len(msg), hexlify(msg[:32]), ' '*10))

In [ ]:
radio.config_gpio(gpio2='TX_STATE', gpio3='RX_STATE')

In [ ]:
p.gpio2

In [ ]:
p['gpio2']